<a href="https://colab.research.google.com/github/ankit-281/Titanic_predictions_deep_learning/blob/main/Deep_Learning_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import numpy as np
import re
import random as rn

from numpy.random import seed
seed(100)
rn.seed(100)
import tensorflow as tf
tf.random.set_seed(100)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
model = Sequential([
    Dense(64, input_shape=(26,), activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
titanic_data_df = pd.read_csv("/content/drive/MyDrive/Datasets/train.csv")

In [38]:
titanic_data_df['Title'] = titanic_data_df['Name'].str.extract(r',\s*([^\.]+)\.')

In [39]:
titanic_data_df = titanic_data_df.drop(columns=['PassengerId', 'Ticket', 'Name'])

In [40]:
titanic_data_df = titanic_data_df.loc[:, titanic_data_df.isna().mean() * 100 <= 50]

In [41]:
titanic_data_df['Embarked'] = titanic_data_df['Embarked'].fillna(titanic_data_df['Embarked'].mode()[0])
titanic_data_df['Age'] = titanic_data_df['Age'].fillna(titanic_data_df['Age'].mean())

In [42]:
titanic_data_df['Family_members'] = titanic_data_df['SibSp'] + titanic_data_df['Parch']

In [43]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
titanic_data_df[['Age', 'Fare', 'Family_members']] = scaler.fit_transform(titanic_data_df[['Age', 'Fare', 'Family_members']])

In [44]:
titanic_data_df = pd.get_dummies(titanic_data_df, columns=['Pclass', 'Sex', 'Title', 'Embarked'], drop_first=True,dtype=int)

In [45]:
titanic_data_df

,Survived,Age,SibSp,Parch,Fare,Family_members,Pclass_2,Pclass_3,Sex_male,Title_Col,...,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Embarked_Q,Embarked_S
0,0,0.271174,1,0,0.014151,0.1,0,1,1,0,...,0,0,1,0,0,0,0,0,0,1
1,1,0.472229,1,0,0.139136,0.1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,0.321438,0,0,0.015469,0.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,0.434531,1,0,0.103644,0.1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,0,0.434531,0,0,0.015713,0.0,0,1,1,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,0.334004,0,0,0.025374,0.0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,1
887,1,0.233476,0,0,0.058556,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
888,0,0.367921,1,2,0.045771,0.3,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
889,1,0.321438,0,0,0.058556,0.0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0


In [46]:
y = titanic_data_df['Survived']

In [47]:
titanic_data_df = titanic_data_df.drop(columns=['Survived'])

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(titanic_data_df, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [49]:
model.compile(optimizer=SGD(), loss='binary_crossentropy', metrics=['accuracy'])

In [56]:
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

In [57]:
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), callbacks=early_stop)

Epoch 1/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6184 - loss: 0.6694 - val_accuracy: 0.6642 - val_loss: 0.6537
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6138 - loss: 0.6564 - val_accuracy: 0.6493 - val_loss: 0.6412
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6002 - loss: 0.6461 - val_accuracy: 0.6493 - val_loss: 0.6301
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6002 - loss: 0.6362 - val_accuracy: 0.6493 - val_loss: 0.6194
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6002 - loss: 0.6261 - val_accuracy: 0.6567 - val_loss: 0.6084
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6002 - loss: 0.6157 - val_accuracy: 0.6567 - val_loss: 0.5974
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6031 - loss: 0.6050 - val_accuracy: 0.6567 - val_loss: 0.5862
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6178 - loss: 0.5938 - val_accuracy: 0.6567 - v

In [58]:
predictions = model.predict(X_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


In [59]:
predictions

array([[0.14764054],
       [0.24041696],
       [0.50521374],
       [0.22120698],
       [0.27631664],
       [0.583514  ],
       [0.14656636],
       [0.14666367],
       [0.14782766],
       [0.14666086],
       [0.5930838 ],
       [0.67440486],
       [0.64932334],
       [0.29290855],
       [0.14640328],
       [0.22684325],
       [0.19983494],
       [0.21920827],
       [0.2138691 ],
       [0.6372519 ],
       [0.386327  ],
       [0.7222308 ],
       [0.24041668],
       [0.63461214],
       [0.14651182],
       [0.1465835 ],
       [0.7407973 ],
       [0.14781827],
       [0.5873088 ],
       [0.14714599],
       [0.23769096],
       [0.71800596],
       [0.5545769 ],
       [0.14810948],
       [0.1466115 ],
       [0.64067143],
       [0.754698  ],
       [0.19389094],
       [0.21821041],
       [0.25892004],
       [0.19974597],
       [0.1456488 ],
       [0.2403841 ],
       [0.63511163],
       [0.3726035 ],
       [0.2079489 ],
       [0.2184218 ],
       [0.702

In [60]:
binary_predictions = (predictions >= 0.5).astype(int)
print(f"Survival rate: {np.mean(binary_predictions)}")

Survival rate: 0.34328358208955223


In [61]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, binary_predictions))
print("\nClassification Report:\n", classification_report(y_test, binary_predictions))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, binary_predictions))

Accuracy: 0.7985074626865671

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.85      0.85        87
           1       0.72      0.70      0.71        47

    accuracy                           0.80       134
   macro avg       0.78      0.78      0.78       134
weighted avg       0.80      0.80      0.80       134


Confusion Matrix:
 [[74 13]
 [14 33]]
